In [1]:
# coding: utf-8
"""
GSDMMを動かすためのコード
"""
from library.filer import Filer
from library.sampleGSDMM import GSDMM
from library.dnp import Evaluation
import glob
import numpy as np
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score, completeness_score, homogeneity_completeness_v_measure, homogeneity_score, normalized_mutual_info_score

In [2]:
list_filepath = glob.glob('files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/*.txt')
list_filepath.sort()
list_testfile = Filer.readdump('./files/rakuten_corpus/rakuten_corpus_master/testfile/list_sepword_label.dump')
list_testword = [row[0] for row in list_testfile]
list_label = [row[1] for row in list_testfile]

outputpath = './files/result/rakuten/experiment4/GSDMM.txt'
removepath = 'files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/'

In [ ]:
for path in list_filepath:
    print path
    path_rev = path.replace(removepath, '')
    Filer.writetxt([path_rev], outputpath)
    for iteration in range(0,10):
        gsdmm = GSDMM(alpha=0.1, beta=0.1, K=100, I=20)
        gsdmm.set_param(path=path)
        list_theta = gsdmm.fit()
        
        array_theta = np.array(gsdmm.list_theta)
        K = len(array_theta[array_theta > 0.0001])
        
        list_predict = []
        for row in list_testword:
            try:
                list_prob = gsdmm.infer(row)
                list_predict.append(np.argmax(list_prob))
            except KeyError:
                list_predict.append(None)

        list_predict_rev = []
        list_label_rev = []
        for predict, label in zip(list_predict, list_label):
            if predict != None:
                list_predict_rev.append(predict)
                list_label_rev.append(label)

        eva = Evaluation()
        dict_result = eva.cal_f_measure(list_predict_rev, list_label_rev)
        
        # NMI
        NMI = normalized_mutual_info_score(list_label_rev, list_predict_rev)
        # vm
        H, C, VM = homogeneity_completeness_v_measure(list_label_rev, list_predict_rev)
        # ARI
        ARI = adjusted_rand_score(list_label_rev, list_predict_rev)
        # AMI
        AMI = adjusted_mutual_info_score(list_label_rev, list_predict_rev)
        
        Filer.writetxt([' '.join([str(dict_result['purity']),str(dict_result['invpurity']),str(dict_result['fvalue']),
                                  str(NMI), str(H), str(C), str(VM), str(ARI), str(AMI), str(K)])],
                       outputpath)

files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/rakuten_preprocessed_005105.txt
finish:  0  iteration
finish: 